# 📖 Agent 實作範例：自動化有聲書生成

本 Notebook 展示如何透過 Agent 串接 **故事 → 圖像 → 語音** 的自動化流程。  
每個步驟都會將輸出保存為檔案，避免重複 Token 消耗，並方便後續流程使用。

---

## 🔹 整體流程
1. 故事內容生成  
2. 圖片內容生成  
3. 語音內容生成  

---

## 1️⃣ 故事內容生成
**輸入**  
- 草稿文字  
- 既有內容 (`.txt` 檔案)  

**處理流程**  
1. 將文字送入 Langserve 服務  
2. 接收生成的故事段落  

**輸出**  
- 將結果存為 `.txt` 檔  
- 避免 Agent 一直傳遞整份故事，降低 Token 消耗  

---

## 2️⃣ 圖片內容生成
**輸入**  
- 最新生成的故事文字  
<!-- - 既有圖片 (`.png`)   -->

**處理流程**  
1. 將圖片編碼為 base64  
2. 文字與圖片送入 Langserve 服務  
3. 接收回傳的圖片（base64 格式）  

**輸出**  
- 將 base64 解碼為二進位資料，存成 `.png` 檔  

---

## 3️⃣ 語音內容生成
**輸入**  
- 最新生成的故事文字  

**處理流程**  
1. 將文字送入 Langserve 服務  
2. 接收回傳的語音（base64 格式）  

**輸出**  
- 將 base64 解碼為二進位資料，存成 `.mp3` 或 `.wav` 檔  

---

## 🔄 流程圖

```mermaid
flowchart TD
    A[故事草稿/舊內容 .txt] --> B[送入 Langserve 生成故事]
    B --> C[故事內容 .txt]
    C --> D[送入 Langserve 生成圖片 (base64)]
    D --> E[解碼並存為 .png]
    C --> F[送入 Langserve 生成語音 (base64)]
    F --> G[解碼並存為 .mp3 / .wav]


## LangServe 服務測試

In [ ]:
import os
import requests

os.chdir("../../")

測試故事生成服務

In [ ]:
response = requests.post(
    "http://localhost:8080/story_telling/invoke",
    json={"input":{'scratch': "Create the first page of the story. This page is about a baby owl capturing a rodent in the night as his dinner",
                   'context': ""}
         }
)

In [ ]:
story_json = response.json()

In [ ]:
story_json['output']

測試影像生成服務

In [ ]:
# import importlib

# image_generation_module = importlib.import_module("tutorial.LLM+Langchain.Week-8.logic.image_generation")
# image_create_pipeline = image_generation_module.image_create_pipeline(image_generation_module.system_template)

In [ ]:
response_image = requests.post(
    "http://localhost:8080/image_generation/invoke",
    json={"input": {'story': story_json['output'],
                    "image_io": []}
    }
)

In [ ]:
response_image.json()['output'].keys()

In [ ]:
response_image.json()['output']['image_base64'][:100]

In [ ]:
response_image.json()['output']['nl_prompt']

In [ ]:
from IPython.display import display, HTML

image_base64 = response_image.json()['output']['image_base64']

HTML(f'<img src="data:image/png;base64,{image_base64}"/>')

測試生成後續後續的故事

In [ ]:
response_next_tory = requests.post(
    "http://localhost:8080/story_telling/invoke",
    json={"input":{'scratch': "Create the next page content following the context",
                   'context': story_json['output']}
         }
)

In [ ]:
next_chapter = response_next_tory.json()['output']

In [ ]:
print(next_chapter)

根據故事和上一張圖片，產生出下一張圖片

透過requests送出base64 string.

本來是想要透過img2img的手法控制圖片風格和內容一致性的，但是發現不確定性很高，所以決定從上一張圖片的prompt下手，看看能不能調高一致性。

In [ ]:
response = requests.post(
    "http://localhost:8080/image_generation/invoke",
    json={"input": {'story': next_chapter + f"\nPrevious image description:\n\n{response_image.json()['output']['nl_prompt']}",
                    # 'image_io': [response_image.json()['output']['image_base64']]
                    'image_io': []
                   }
    }
)

In [ ]:
response.json()['output']['nl_prompt']

In [ ]:
image_base64 = response.json()['output']['image_base64']

HTML(f'<img src="data:image/png;base64,{image_base64}"/>')

測試語音生成

In [ ]:
response = requests.post(
    "http://localhost:8080/audio_generation/invoke",
    json={"input": {'input': "How are you doing?"}}
)

In [ ]:
import base64
from IPython.display import Audio

audio_bytes = base64.b64decode(response.json()['output'])

# 在 notebook 中播放
Audio(audio_bytes, autoplay=True)

# with open("tutorial/LLM+Langchain/Week-8/test_sample.mp3", "wb") as f:
#     f.write(audio_bytes)


## 生成工具模板

In [ ]:
from textwrap import dedent
from typing import Optional, Any, List, Tuple, Callable
from pathlib import Path

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain.tools import BaseTool
from langchain_core.runnables import Runnable
from pydantic import BaseModel, Field
from pydantic import FilePath

from initialization import credential_init


class ToolTemplate(BaseTool):

    """
    ToolTemplateHTTP: 一個專門用來呼叫 Langserve REST API 的 Agent Tool

    - 使用 PydanticOutputParser 保證輸入格式正確
    - 支援多欄位 input/output 處理器
    - 對 API 呼叫加上錯誤處理
    """
    
    runnable: str = Field(..., description='The Langserve endpoint')
    name: str
    input_parser: PydanticOutputParser
    description: str
    input_data_processors: Optional[List[Tuple[str, Callable[[Any], Any]]]] = None
    output_data_processors: Optional[List[Tuple[Optional[str], Callable[[Any], Any]]]] = None
    
    @classmethod
    def create(cls, runnable: str, name: str, description_template: str,
               input_parser: PydanticOutputParser, input_data_processors: Optional=None,
               output_data_processors: Optional=None):

        """建立 Tool 實例，會自動把輸入格式需求加入 description"""
        
        input_format_instruction = input_parser.get_format_instructions()
        
        description = description_template.format(
            input_format_instruction=input_format_instruction
        )
        
        return cls(runnable=runnable, name=name, description=description,
                   input_parser=input_parser, input_data_processors=input_data_processors,
                   output_data_processors=output_data_processors)
    
    def _run(self, **input_):

        """執行 Tool，同步版本"""

        print(f"Call tool: {self.name}")
        
        runnable_inputs = input_

        # 1. input processors（前處理）
        if self.input_data_processors:
            for field, fn in self.input_data_processors:
                if field in runnable_inputs:
                    runnable_inputs[field] = fn(runnable_inputs[field])
            
        # 2. 呼叫 Langserve REST API
        try:
            response = requests.post(
                str(self.runnable),
                json={"input": runnable_inputs},
                timeout=60,
            )
            response.raise_for_status()
            result = response.json()
        except Exception as e:
            raise RuntimeError(f"Langserve call failed: {e}, inputs={runnable_inputs}")

        if "output" not in result:
            raise RuntimeError(f"Invalid response format from Langserve: {result}")
        
        output = result['output']

        # 4. update the state varaibles:
        for key in session_state.keys():
            if key in output:
                session_state[key] = output[key]
        
        # 5. output processors（後處理）
        if self.output_data_processors:
            for field, fn in self.output_data_processors:
                if not field:
                    fn(output, runnable_inputs['filename'])
                else:
                    fn(output[field], runnable_inputs['filename'])
                    
        # 預設回傳「檔名」如果有 filename，否則回傳輸出的字串
        return runnable_inputs.get("filename", output)

    def _arun(self, query: str):

        raise NotImplementedError("This tool does not support async")

### State Variables

In [ ]:
session_state = {}

session_state['nl_prompt'] = None

In [ ]:
from pathlib import Path
from textwrap import dedent
from typing import List

from pydantic import BaseModel, Field


class StoryInput(BaseModel):
    scratch: str = Field(description=dedent("""\
                                            The draft, notes, or rough idea for the current page of the story.
                                           （故事當前頁面的草稿、筆記或初步構想
                                            """))
    context: List[FilePath] = Field(default_factory=list, description=dedent("""\
                                                              A list of previously generated .txt files that contain story content.  
                                                              Used to maintain narrative consistency and continuity across images.  
                                                              先前生成的 .txt 檔案清單，其中包含故事內容。  
                                                              用於保持影像生成過程中的敘事一致性與連貫性。
                                                              """))
    filename: str = Field(..., description=dedent("""\
                                             The file path where the generated story text will be saved.
                                            （生成的故事文本將被儲存的檔案路徑）
                                             """))


def export_to_txt(text, filename: Path):

    """
    將文字儲存為檔案。這樣子agent就可以靠傳遞名稱來記得前面的訊息，而不是直接將所有的訊息傳遞給下去。
    靠著犧牲I/O的效率換取節省Tokens的費用
    """
    print(f"export file to {filename}")
    
    dir_ = Path(filename).parent

    if not os.path.isdir(dir_):
        os.makedirs(dir_)
    
    with open(filename, "w", encoding="utf-8") as file:
        file.write(text)


def read_from_txt(filename) -> str:

    with open(filename, "r", encoding="utf-8") as file:
        content = file.read()

    return content


def read_from_list_of_text(filenames) -> str:

    """
    讀取所有的檔案，還獲得之前產生的訊息
    """
    
    return "\n\n".join([read_from_txt(f) for f in filenames])


story_input_data_processors = [("context", read_from_list_of_text)]

# ===================================================================
"""
if self.input_data_processors:
    for field, fn in self.input_data_processors:
        if field in runnable_inputs:
            runnable_inputs[field] = fn(runnable_inputs[field])

邏輯:
1. 查詢 input_data_processors 這個物件是否存在
2. 若是存在，將Field 和 函數全部展開。若是 field 存在於 input 中，使用對應的函數處理該Field
"""
# ===================================================================


story_output_data_processors = [(None, export_to_txt)]

story_telling_tool = ToolTemplate.create(
    runnable="http://localhost:8080/story_telling/invoke",
    name="Story-generation-tool",
    description_template=dedent("""\
                                This tool is designed to generate a story one page at a time.
                                Provide a draft or idea for the current page (`scratch`), along with 
                                the preceding story context stored as .txt files (`context`), 
                                and specify where the generated text should be saved (`filename`).
                                    
                                Input format: {input_format_instruction}
                                """
                                ),
    input_parser=PydanticOutputParser(pydantic_object=StoryInput),
    output_data_processors = story_output_data_processors,
    input_data_processors = story_input_data_processors
)

### 影像生成

In [ ]:
class ImageInput(BaseModel):
    story: str = Field(description=dedent("""\
                                          The file name of the current chapter, whose content will be used to generate the image prompt.  
                                          當前章節的檔案名稱，其內容會被用於影像生成。
                                          """)
                      )
    # FilePath ensures the input path exists and is a valid file
    image_io: List[str] = Field([], description=dedent("""\
                                                   Path to the previously generated image.  
                                                   Used in img2img generation to maintain visual and texture consistency.  
                                                   先前生成影像的路徑。  
                                                   在 img2img 生成中用於保持視覺與材質的一致性。
                                                   """)
                                    )
    # Path allows flexibility: file may not exist yet but must be a valid path object.
    filename: str = Field(..., description=dedent("""\
                                                  Destination file path where the generated image will be saved.  
                                                  生成影像的儲存檔案路徑。
                                                  """)
                          )


def image_to_base64(image_path):
    
    with Image.open(image_path) as image:
        
        # Save the Image to a Buffer
        buffered = io.BytesIO()
        image.save(buffered, format="JPEG")
        
        # Encode the Image to Base64
        image_str = base64.b64encode(buffered.getvalue())
    
    return image_str.decode('utf-8')


def image_to_base64_from_list(filenames: List[str]) -> List[Optional[str]]:

    # return [image_to_base64(f) for f in filenames]
    return []


def export_to_image(content: str, filename: str):

    """
    將base64字串(content)儲存在檔案filename中
    """
    
    dir_ = Path(filename).parent

    if not os.path.isdir(dir_):
        os.makedirs(dir_)
    
    image_bytes = base64.b64decode(content)
    
    with open(filename, "wb") as fh:
        fh.write(image_bytes)

    
def story_adaptation(story: str):

    story = read_from_txt(story)
    
    nl_prompt = session_state['nl_prompt']
    
    if nl_prompt:
        story += f"\nPrevious image description:\n\n{nl_prompt}"
    
    return story
    


image_output_data_processors = [('image_base64', export_to_image)]

image_input_data_processors = [("story", story_adaptation),
                                ("image_io", image_to_base64_from_list)]

image_tool = ToolTemplate.create(
    runnable="http://localhost:8080/image_generation/invoke",
    name="Image-generation-tool",
    description_template=dedent("""\
                                This tool is designed to generate an image to the correspoinding narrative.
                                Provide a narrative (`story`), along with 
                                the preceding images stored as .png files (`image_io`), 
                                and specify where the generated text should be saved (`filename`).
                                    
                                Input format: {input_format_instruction}
                                """
                                ),
    input_parser=PydanticOutputParser(pydantic_object=ImageInput),
    output_data_processors = image_output_data_processors,
    input_data_processors = image_input_data_processors
)

### 語音生成

In [ ]:
class AudioInput(BaseModel):
    input: str = Field(description=dedent("""\
                                          The file name of the current chapter, whose content will be used to generate the audio content with text to sound (TTS).  
                                          當前章節的檔案名稱，其內容會被用於TTS文字轉語音。
                                          """)
                                    )
    # Path allows flexibility: file may not exist yet but must be a valid path object.
    filename: str = Field(..., description=dedent("""\
                                                  Destination mp3 file path where the generated audio will be saved.  
                                                  mp3語音檔的儲存檔案路徑。
                                                  """)
                         )

def export_to_audio(content, filename):

    dir_ = Path(filename).parent

    if not os.path.isdir(dir_):
        os.makedirs(dir_)
    
    audio_bytes = base64.b64decode(content)
    
    with open(filename, "wb") as fh:
        fh.write(audio_bytes)


audio_output_data_processors = [(None, export_to_audio)]

# 直接借用 story telling 的函數
"""
audio_generation 中 使用 `input`這個變數

def tts_synthesizer(kwargs):
    
    response = client.audio.speech.create(
    model=model,
    voice=voice,
    input=kwargs['input'],
    instructions=instructions)

    # Get audio bytes
    audio_data = response.read()

    # Encode to base64 for JSON serialization
    audio_b64 = base64.b64encode(audio_data).decode("utf-8")

    return audio_b64
"""
audio_input_data_processors = [('input', read_from_txt)]

audio_tool = ToolTemplate.create(
    runnable="http://localhost:8080/audio_generation/invoke",
    name="Audio-generation-tool",
    description_template=dedent("""\
                                This tool is designed to generate an .mp3 file to a corresponding narrative.
                                Provide a narrative (`story`), along with 
                                and specify where the generated audio should be saved (`filename`).
                                    
                                Input format: {input_format_instruction}
                                """
                                ),
    input_parser = PydanticOutputParser(pydantic_object=AudioInput),
    output_data_processors = audio_output_data_processors,
    input_data_processors = audio_input_data_processors
)

In [ ]:
from textwrap import dedent

from langchain.agents import create_agent

system_prompt = dedent("""\
You are a helpful AI assistant responsible for generating high-quality audiobook content page by page.

    For each page:

    1. Start from the draft text provided by the user.
       - Improve, expand, or rewrite it as needed while preserving the story’s intent and tone.

    2. Produce three outputs for that page:
       - Text content for the page (TXT).
       - A corresponding image (PNG).
       - A corresponding audio narration (MP3).

    3. You MUST call Story-generation-tool before Audio-generation-tool and Image-generation-tool
        
    4. Save all three files—the page text (.txt), the generated image (.png), and the audio narration (.mp3)—using the same base filename inside the directory:
       tutorial/week_8/story_t est

    5. Ensure filename consistency across formats, for example:
       - page_01.txt
       - page_01.png
       - page_01.mp3

    Your role is to reliably produce well-written content, matching visuals, and clear audio files for each page in sequence.
""")


tools = [story_telling_tool, image_tool, audio_tool]

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o", temperature=0, 
                  )

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt,
    name = "audio_book_agent"
)

In [ ]:
import base64

from langchain.messages import HumanMessage

session_state = {}

session_state['nl_prompt'] = None

query = dedent("""
Create a chapter of a baby owl capturing a rodent in the night as his dinner.
""")

agent.invoke({"messages": HumanMessage(content="Create a chapter of a baby owl capturing a rodent in the night as his dinner.")})

成功的生成了一頁的內容，Agent可以幫我們生成整個故事嗎?

In [ ]:
import shutil

shutil.rmtree("tutorial/week_8/story_test")

In [ ]:
result = agent.invoke({"messages": HumanMessage(content="Create an 4 pages story for a 4-years-old child. He likes snow owl.")})

In [ ]:
result

## 🎧 互動式有聲書內容生成

- 不一定需要完整的 **Agent 架構**，因為流程的每一步（故事 → 圖像 → 語音）都已經明確定義，能由使用者主動觸發。  
- 可以直接基於 **聊天機器人** 的互動形式進行，每次輸入使用者的需求或指令後，系統依照指定步驟生成對應內容。  
- 使用者可以在故事生成過程中即時調整方向，例如指定角色、情節走向或語氣，提升 **客製化體驗**。  
- 這種互動方式非常適合 **語言學習** 場景：  
  - 學習者能一邊閱讀故事、一邊聽有聲輸出  
  - 可即時修改故事情節，產生更貼近學習需求的內容  
  - 搭配圖片與語音，提升沉浸式學習效果  

# 🧩 LangGraph 框架與計劃-行動代理人簡介

## 一、LangGraph 是什麼？

**LangGraph** 是一個專門為構建 **多智能體系統（Multi-Agent Systems）** 而設計的框架，提供了結構化的方式來設計、協調與管理多個智能體之間的互動。  
它建立在 **LangChain** 的概念之上，並引入了「圖（Graph）」的思維模型：  
每個智能體（Agent）被視為圖中的一個節點（Node），節點之間的邊（Edge）代表任務流或資訊交換的關係。

### 🌟 LangGraph 的核心特點

- 🔗 **節點導向設計**：每個節點可代表一個智能體、工具或決策模組。  
- 🔄 **靈活的任務流程**：支援有條件的任務轉移（例如根據上下文或任務狀態動態選擇不同智能體）。  
- 🧠 **模型互補性**：可同時使用多種大型語言模型（LLMs）或不同的外部工具。  
- 📊 **可視化工作流**：開發者可以直觀地觀察智能體之間的互動與任務流向。  
- 🧩 **有狀態的任務控制**：支援任務回溯、重試與流程中斷恢復等功能。  


---

## 二、LangChain vs. LangGraph

在現代 LLM 應用開發中，**LangChain** 和 **LangGraph** 是兩個常被提及的框架。  
它們皆旨在簡化與大型語言模型的整合流程，但在設計理念與使用場景上有所不同。

| 特點 | LangChain | LangGraph |
|------|-----------|-----------|
| 架構風格 | 函式導向（Function-based） | 圖形導向（Graph-based） |
| 核心用途 | 組合不同的工具與鏈（Chains）以處理任務 | 定義狀態轉換與流程控制的狀態機 |
| 適合場景 | 線性流程、多工具串接、單一任務代理人 | 有狀態的工作流程、多步驟決策、動態流程切換 |
| 控制流程能力 | 較弱，需透過程式邏輯控制 | 較強，內建狀態管理與動態節點切換 |
| 易學程度 | 相對容易上手 | 初期需理解狀態機與圖結構概念 |
| 通用性 | 模組多、通用性強 | 更適合複雜與長期任務管理 |

> 💡 **簡而言之**：  
> - **LangChain** 像是一個「積木箱」，提供各種模組供你自由組合。  
> - **LangGraph** 則像是一個「流程編輯器」，讓你清楚定義任務節點與流程邏輯。

---

## 三、為什麼選擇 LangGraph？

LangGraph 建立在 LangChain 的基礎上，但提供了更明確的 **流程控制能力**。  
透過「有向圖（Directed Graph）」定義節點（Node）與狀態轉移（Edge），開發者可以輕鬆設計出具備以下特性的 LLM 應用：

- ✅ **記憶狀態（Stateful）**：能保存上下文與任務進度。  
- 🔁 **可回溯（Reversible）**：支援任務失敗時的重試與回溯。  
- ⚡ **動態流程（Dynamic Flow）**：根據結果或上下文動態切換下一步節點。  
- 🧩 **多智能體協作（Multi-Agent Coordination）**：支援多個智能體協同工作。  

這使得 LangGraph 特別適合需要長期規劃、決策、以及多階段任務的應用場景，例如：
- AI 助理工作流程管理  
- 自動化決策系統  
- 複雜任務規劃與執行（如科研助理、專案管理代理人）

---

## 四、計劃-行動代理人（Plan-Action Agent）

### 🔍 什麼是計劃-行動代理人？

計劃-行動代理人是一種基於「**計劃（Plan）** → **行動（Action）** → **反饋（Feedback）**」迭代循環的智能體。  
它會根據環境狀況生成計劃，並逐步執行與調整，直到達成目標。

典型的工作流程如下：

1. **觀察環境**：收集當前環境狀態或上下文。  
2. **計劃生成**：根據觀察結果生成一系列行動計劃。  
3. **行動執行**：逐步執行每個計劃步驟。  
4. **反饋與調整**：根據執行結果與環境變化調整計劃，確保最終目標的達成。

---

## 五、在 LangGraph 中實現計劃-行動代理人

LangGraph 為構建這類代理人提供了理想的結構。  
在 LangGraph 中，我們可以透過「節點」定義不同的智能體角色（如計劃生成器、執行者、監督者），並以「邊」定義他們之間的互動流程。

例如：

[觀察節點] → [計劃生成節點] → [行動執行節點] → [反饋節點]

↘———————————————↗（根據反饋動態回到計劃生成）


這樣的設計允許代理人在任務執行中進行：
- 任務重試（retry）  
- 動態決策（dynamic routing）  
- 自適應策略調整（adaptive planning）  

---

## 六、接下來的內容

接下來，我們將透過實作一個 **計劃-行動代理人（Plan-Action Agent）** 的範例，  
展示如何使用 LangGraph 來：

- 定義多個智能體節點  
- 設計任務流程與條件轉移  
- 管理狀態與任務回溯  

透過這個實作，你將能理解 LangGraph 在 **多智能體協作**、**動態決策** 與 **有狀態任務控制** 方面的強大能力。


In [ ]:
import os

os.chdir("../../")

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser

from initialization import credential_init

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-mini", 
                   temperature=0
                  )

In [ ]:
import operator
from typing import Annotated, List, Tuple, Union, Optional
from typing_extensions import TypedDict

# State，把這個想像成是Agent的短期記憶
class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: List[Tuple]
    response: str

In [ ]:
from typing import List, Literal, Union
from textwrap import dedent
from pydantic import BaseModel, Field

from langchain_core.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate


class Plan(BaseModel):
    """
    Plan to follow in the future
    """
    type: Literal['plan'] = 'plan'
    steps: List[str] = Field(description="different steps to follow, should be in sorted order")


planner_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
        dedent("""\
        For the given objective, come up with a simple step by step plan.
        This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps.
        The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.
        """)),
        ("placeholder", "{messages}")
    ]
)

model_planner = model.with_structured_output(Plan)

In [ ]:
planner = planner_prompt|model_planner

In [ ]:
planner.invoke(
    {
        "messages": [
            ("user", "What is the hometown of the current Australia open winner")
        ]
    }
)

In [ ]:
class Response(BaseModel):
    """Response to user."""
    type: Literal['response'] = 'response'
    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description=dedent("""\
        Action to perform. If you want to respond to user, use `response`.
        If you need to further use tools to get the answer, use `plan`.
        """
    )
    )

# inpput: 最終的目標 (goal)
# plan: 目前需要完成的任務 (task)
# past_steps: 過去已經完成的任務

replanner_prompt = ChatPromptTemplate.from_template(
    dedent("""For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
))

model_act = model.with_structured_output(Act)

replanner = replanner_prompt | model_act

In [ ]:
replanner.invoke({"plan": "Identify the current Australian Open winner for the most recent tournament.",
                  "past_steps": [],
                  "input": "What is the hometown of the current Australia open winner"})

In [ ]:
# !pip install langgraph

In [ ]:
import asyncio

from openai import OpenAI, AsyncOpenAI
from langchain.tools import BaseTool
from langchain_core.output_parsers import PydanticOutputParser

client = OpenAI()
async_client = AsyncOpenAI()


class Inputs(BaseModel):
    query: str = Field(description="User query")
    country_code: str = Field(description="ISO 3166-1 alpha-2 code of the user language")


class SearchTool(BaseTool):

    input_output_parser: PydanticOutputParser = PydanticOutputParser(pydantic_object=Inputs)
    input_format_instructions: str = input_output_parser.get_format_instructions()
    
    name:str = "websearch-tool"
    description_template:str = dedent("""
    Currently it is 2026.    
    Use this tool to collect information from the internet, when you are not sure you know the answer.
    The input contains the user's question `query` and the ISO 3166-1 alpha-2 `country_code` inferred from the user's language.
    input format instructions: {input_format_instructions}
    """)

    description: str = description_template.format(input_format_instructions=input_format_instructions)

    def _build_messages_and_opts(self, input_):
        """Shared logic for sync + async"""
        
        query = input_["query"]
        country_code = input_["country_code"]

        tool = {"type": "web_search",
                         "user_location":{
                             "type": "approximate",
                             "country": country_code,
                         },
                        "search_context_size": "medium"
                        }
        return query, tool
        
    def _run(self, **input_):
        
        query, tool = self._build_messages_and_opts(input_)

        response = client.responses.create(
                    model="gpt-4o-mini",
                    tools=[tool],
                    tool_choice="auto",
                    input=query)

        
        return response.output_text
    
    async def _arun(self, **input_):
        
        query, tool = self._build_messages_and_opts(input_)

        response = await async_client.responses.create(
            model="gpt-4o-mini",
            tools=[tool],
            tool_choice="auto",
            input=query
        )

        return response.output_text

In [ ]:
from typing import Literal

from langchain.agents import create_agent

agent = create_agent(name='websearch_agent',
                     model='gpt-4o-mini',
                     tools=[SearchTool()])

### Define the functionalities of the nodes

In [ ]:
async def execute_step(state: PlanExecute):
    """
    此方法負責執行目前計劃（plan）中的第一個步驟。
    在 LangGraph 節點中，它通常作為「任務執行節點」，負責根據計劃指令呼叫代理模型（agent）來執行具體任務，並將執行結果記錄到狀態（state）中。
    
    主要邏輯：
    
    從 state 讀取目前的計劃 (plan)。
    
    以編號方式將整個計劃格式化成可讀字串。
    
    取出計劃中的第一個步驟作為要執行的任務。
    
    呼叫 agent.ainvoke()，讓智能代理執行該任務。
    
    將執行結果與任務名稱以 tuple 形式存入 past_steps。
    
    若 state 中尚無過去紀錄，建立新的 past_steps；否則將新結果附加到現有紀錄中。
    
    參數：
    
    state (PlanExecute)：包含計劃、過往步驟與當前輸入的執行狀態。
    
    回傳：
    
    dict：包含更新後的 past_steps，即執行任務的歷史紀錄。
    
    應用情境：
    此節點可用於多步驟任務執行流程，例如：
    
    根據計劃逐步執行動作（如資料蒐集、分析、報告撰寫等）。
    
    在多代理系統中由特定 agent 執行分配任務。
    """
    plan = state["plan"]
    plan_str = "\n".join(f"{i + 1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""For the following plan:{plan_str}\n\nYou are tasked with executing step {1}, {task}."""
    
    agent_response = await agent.ainvoke(
        {"messages": [{"role": "user", "content": task_formatted}]}
    )
    
    current_step = [(task, agent_response['messages'][-1].content)]

    # push the result to state
    if "past_steps" not in state:
        return {"past_steps": current_step}
    else:
        return {"past_steps": state["past_steps"] + current_step}

In [ ]:
async def plan_step(state: PlanExecute):
    """
    此方法負責根據使用者輸入建立完整的計劃（plan）。
    在 LangGraph 節點中，它通常作為「計劃生成節點」，讓規劃器（planner）根據使用者指令產生一系列可執行步驟。
    
    主要邏輯：
    
    呼叫 planner.ainvoke()，將使用者輸入的訊息傳入規劃模型。
    
    接收規劃器輸出的步驟（plan.steps）。
    
    將這些步驟存入 state，以便後續節點執行。
    
    參數：
    
    state (PlanExecute)：包含使用者輸入的執行狀態（state["input"] 為用戶的主要指令）。
    
    回傳：
    
    dict：包含一個鍵值對 { "plan": plan.steps }，即模型生成的任務步驟清單。
    
    應用情境：
    此節點可用於任務導向系統的開頭階段，例如：
    
    將「撰寫市場報告」分解為「資料蒐集 → 整理 → 分析 → 撰寫」。
    
    在多階段推理或自動化任務中生成具體工作計劃。
    """

    # push the result to state
    plan = await planner.ainvoke({"messages": [("user", state["input"])]})
    return {"plan": plan.steps}

In [ ]:
async def replan_step(state: PlanExecute):

    """
    此方法負責在任務執行過程中進行動態重新規劃（Replanning）。
    當代理執行中遇到問題、條件改變或4任務未完成時，該節點可呼叫「再規劃器」（replanner）生成新的計劃或直接輸出最終回應。
    
    主要邏輯：
    
    呼叫 replanner.ainvoke()，將當前執行狀態作為輸入。
    
    若返回結果包含 Response 類型的動作，表示可直接輸出最終回覆。
    
    若結果為新的計劃步驟，則更新 plan。
    
    若無新步驟可執行，則使用最後一次執行的結果作為回應。
    
    參數：
    
    state (PlanExecute)：包含目前的執行狀態、歷史步驟與已知計劃。
    
    回傳：
    
    dict：包含新的 plan 或最終 response。
    
    應用情境：
    此節點用於：
    
    執行任務時根據失敗步驟或環境變化調整計劃。
    
    讓智能體具備「自我修正」能力。
    
    在長程任務中根據中間輸出重新安排後續步驟。
    """

    # 若是 replanner 給出 Response, 結束迴圈
    # 若是 replanner 給出 Aaction, 繼續執行
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        if output.action.steps:
            return {"plan": output.action.steps}
        else:
            return {"response": state['past_steps'][-1][1]}

In [ ]:
def should_end(state: PlanExecute):
    """
    此方法負責決定工作流程是否結束。
    在 LangGraph 節點中，它通常是條件分支（conditional edge）的判斷邏輯，用來檢查任務是否完成或是否應該回到代理繼續執行。
    
    主要邏輯：
    
    若 state 中存在 response 且其值非空，代表任務已完成 → 回傳 END。
    
    否則，回傳 "agent"，表示仍需由代理節點繼續執行。
    
    參數：
    
    state (PlanExecute)：目前的執行狀態。
    
    回傳：
    
    END（任務完成，結束節點）或 "agent"（繼續執行下一階段）。
    
    應用情境：
    此節點常用於：
    
    流程控制（決定任務是否結束）。
    
    多節點 LangGraph 流程中設置終止條件。
    
    確保系統在任務完成後適時停止，不進行多餘操作。
    """
    
    if "response" in state and state["response"]:
        return END
    else:
        return "agent"

LangGraph 是宣告式（declarative）的，而非純粹動態（purely dynamic）的。
這表示當你在建立工作流程圖（workflow graph）時（也就是在執行之前），LangGraph 需要事先知道所有可能的分支——
即使你的條件函式（should_end）會在執行階段才決定實際要走哪一個分支。

In [ ]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(PlanExecute)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("agent", execute_step)

# Add a replan node
workflow.add_node("replan", replan_step)

workflow.add_edge(START, "planner")

# From plan we go to agent
workflow.add_edge("planner", "agent")

# From agent, we replan
workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    should_end,
    ["agent", END]
)

app = workflow.compile()

In [ ]:
from IPython.display import Image, display

display(Image(app.get_graph(xray=True).draw_mermaid_png()))

#### 範例一: 2023年澳洲網球公開賽冠軍的老家在哪裡?

In [ ]:
config = {"recursion_limit": 50}
inputs = {"input": "what is the hometown of the mens 2023 Australia open winner?"}

In [ ]:
# mlflow server --host 127.0.0.1 --port 8080
import mlflow

experiment = "Week-8-Exp1"
uri = "http://127.0.0.1:8081"

mlflow.set_tracking_uri(uri=uri)

# Start or get an MLflow run explicitly
mlflow.set_experiment(experiment)

mlflow.langchain.autolog()

In [ ]:
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

#### 範例二 台灣職棒富邦悍將的啦啦隊三本柱是誰?

In [ ]:
config = {"recursion_limit": 50}
inputs = {"input": "台灣職棒富邦悍將的啦啦隊三本柱是誰?"}

In [ ]:
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

## 💬 評論：LangGraph 任務執行觀察4與優化建議

這次的最終答案是正確的——**「富邦悍將啦啦隊的三本柱為李珠珢、李雅英、南珉貞」**，結果符合事實。  
不過整體推理與行動過程顯得有些「笨拙」，存在明顯的優化空間。

---

### 🧠 問題觀察
1. **重複與低效的查詢步驟**  
   多次執行類似的 `websearch`，查詢關鍵字差異不大，導致系統重複獲取相近內容，浪費資源與時間。  

2. **上下文理解不足**  
   當已經在前一步確定「三本柱」指的是富邦悍將的韓籍啦啦隊成員時，後續步驟仍然重新查詢「三位成員的名字」，  
   顯示模型沒有有效利用先前的狀態（`past_steps`）。  

---

### ⚙️ 改進建議
1. **強化上下文記憶使用（State Management）**  
   - 在 `execute_step` 或 `replan_step` 階段中，明確讓 agent 檢查 `past_steps`，避免重複查詢相同問題。  

2. **加入動態停止條件（Dynamic Stop Condition）**  
   - 若 `agent_response` 已包含高置信度的具體名稱（例如三個人名），可直接將狀態切換為 `END`，無需繼續循環。  

3. **優化搜尋策略**  
   - 將多步查詢整合為單一明確查詢，例如：  
     「富邦悍將 啦啦隊 三本柱 2025 成員名單」，以減少噪音。  

4. **修正輸出格式生成邏輯**  
   - 在 LLM Prompt Template 中，使用結構化格式（如 JSON 或 YAML）明確定義 `Thought / Action / Observation` 區塊，減少格式錯誤。  

5. **可考慮加入 Re-Planning 判斷強化**  
   - 當系統偵測到重複查詢或內容無變化時，可自動觸發 `replan_step`，切換策略或直接輸出結果。  

---

### ✅ 總結
整體而言，系統最終得出了正確答案，說明任務設計邏輯有效；  
但在過程中表現出缺乏上下文記憶、重複查詢與格式錯誤等問題。  

若能加強狀態判斷、結果置信度評估與查詢整合，將能顯著提升效率與智能程度。

### Reserved / Special Parts of `config`

Below are the reserved or commonly used keys in the `config` dictionary when working with **LangGraph**:

| Key | Purpose | Example |
|------|----------|----------|
| `"configurable"` | ✅ **Main section for user-defined and framework config.** Used by checkpointers, node settings, etc. | `{"configurable": {"thread_id": "abc123"}}` |
| `"recursion_limit"` | **Limits graph recursion depth.** Prevents infinite loops or too-deep graph calls. | `{"recursion_limit": 10}` |


# LangGraph Memory

## configurable

- 取得工作流的最後一個state
- 根據當前的state繼續執行

Dummy Example

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver

class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]

def node_a(state: State):
    return {"foo": "a", "bar": ["a"]}

def node_b(state: State):
    return {"foo": "b", "bar": ["b"]}


workflow = StateGraph(State)
workflow.add_node("node_a", node_a)
workflow.add_node("node_b", node_b)
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

"""
Checkpoints:

The state of a thread at a particular point in time is called a checkpoint. 
Checkpoint is a snapshot of the graph state saved at each superstep and is represented by StateSnapshot object.
"""

# 要取得state的內容，你必須要同時有 InMemeorySaver 和 configurable
checkpointer = InMemorySaver()
config = {"configurable": {"thread_id": "1"}}

graph = workflow.compile(checkpointer=checkpointer)
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
graph.invoke({"foo": ""}, config)

## Get state

當與已儲存的圖形狀態互動時，您必須指定一個執行緒識別碼（thread identifier）。您可以透過呼叫 graph.get_state(config) 來查看圖形的最新狀態。此方法會回傳一個 StateSnapshot 物件，該物件對應於在 config 中提供的執行緒 ID 所關聯的最新檢查點（checkpoint），或者，如果提供了檢查點 ID，則會回傳與該執行緒相關的檢查點狀態。

In [ ]:
state = graph.get_state(config)
print(state)

## Get state history

您可以透過呼叫 graph.get_state_history(config) 來取得指定執行緒的完整圖形執行歷史。此方法會回傳一個與 config 中提供的執行緒 ID 相關聯的 StateSnapshot 物件清單。值得注意的是，這些檢查點（checkpoints）會依時間順序排列，最新的檢查點／StateSnapshot 會位於清單的最前端。

In [ ]:
config = {"configurable": {"thread_id": "1"}}
list(graph.get_state_history(config))

## Get State: specific to checkpoint_id

In [ ]:
state = graph.get_state({"configurable": {"thread_id": "1",
                                          "checkpoint_id": '1f0c7703-9870-6223-8001-9f6b6613e6e7'}})
print(state)

## Replay

也可以重播先前的圖形執行過程。若我們在呼叫圖形時同時指定 thread_id 和 checkpoint_id，系統將會重新執行對應於該 checkpoint_id 檢查點之前的所有步驟，並僅執行該檢查點之後的步驟。

- thread_id 是執行緒的識別碼。
- checkpoint_id 是指向該執行緒中某個特定檢查點的識別碼。

In [ ]:
graph.invoke(None, config={"configurable": {"thread_id": "1",
                                            "checkpoint_id": '1f0c7703-9870-6223-8001-9f6b6613e6e7'}})

重要的是，LangGraph 具備辨識特定步驟是否已被執行過的能力。若某個步驟已經執行過，LangGraph 會在圖形中重播該步驟，而不會重新執行它——但這僅適用於在所提供的 checkpoint_id 之前的步驟。至於 checkpoint_id 之後的所有步驟，則都會被重新執行（即產生一個新的分支），即使這些步驟先前已經執行過。

## Resume

In [ ]:
graph.invoke({"foo": "c"}, config)

## InMemoryStore

In [ ]:
from langgraph.store.memory import InMemoryStore

in_memory_store = InMemoryStore()

記憶（Memories）是以一個元組（tuple）作為命名空間（namespace）來區分的，在此特定範例中為 (<user_id>, "memories")。命名空間的長度可以是任意的，並且可以代表任何內容，不一定必須與使用者相關。

In [ ]:
user_id = "1"
namespace_for_memory = (user_id, "memories")

In [ ]:
import uuid

memory_id = str(uuid.uuid4())

memory = {"food_preference" : "I like pizza"}

# 在特定用戶，特定主題之下，將特定內容用 memory_id 標記並且儲存
in_memory_store.put(namespace_for_memory, memory_id, memory)

### 檢索

根據用戶ID和主題來搜尋相關紀錄

In [ ]:
in_memory_store.search(namespace_for_memory)

>- value：此記憶的內容（本身為一個字典）
>- key：此命名空間中該記憶的唯一鍵值
>- namespace：由字串組成的清單，表示此記憶類型所屬的命名空間
>- created_at：此記憶的建立時間戳記
>- updated_at：此記憶的更新時間戳記

加入語意搜索

In [ ]:
from langchain.embeddings import init_embeddings

In [ ]:
embeded = init_embeddings("huggingface:BAAI/bge-m3")

In [ ]:
len(embeded.embed_query("Hello World"))

The main configuration options:

>- embed：嵌入（embedding）提供者（例如 "openai:text-embedding-3-small"），或是自訂函式的路徑（doc）。可用的 provider:model 組合取決於 LangChain 的支援。
>- dims：所選嵌入模型的維度大小（例如 OpenAI 的 text-embedding-3-small 為 1536 維）。
>- fields：要建立索引的欄位清單。可使用 ["$"] 來索引整份文件，或指定 JSON 路徑，例如 ["text", "summary", "messages[-1]"]。

In [ ]:
store = InMemoryStore(
    index={
        "embed": embeded,  # Embedding provider
        "dims": 1024,                              # Embedding dimensions
        "fields": ["food_preference"]         # Fields to embed
    }
)

In [ ]:
store.put(namespace_for_memory, memory_id, memory)

store.put(
    namespace_for_memory,
    str(uuid.uuid4()),
    {
        "food_preference": "I love Italian cuisine",
        "context": "Discussing dinner plans"
    },
    index=["food_preference"]  # Only embed "food_preferences" field
)

store.put(
    namespace_for_memory,
    str(uuid.uuid4()),
    {"system_info": "Last updated: 2024-01-01"},
    index=False
)

In [ ]:
store.search(
    namespace_for_memory,
    query="What does the user like to eat?",
    limit=5  # Return top 3 matches
)

### Search with Filter

In [ ]:
store.search(namespace_for_memory, filter={"context": "Discussing dinner plans"})

在Workflow中調用store

We can access the in_memory_store and the user_id in any node by passing store: BaseStore and config: RunnableConfig as node arguments. Here's how we might use semantic search in a node to find relevant memories

照著官方文件做，會報錯。

In [ ]:
from langchain_core.runnables.config import RunnableConfig
from langchain_core.stores import BaseStore

class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]

def node_a(state: State, config: RunnableConfig):

    print(config["configurable"]["user_id"])

    print("*****")
    print(config['configurable']['__pregel_runtime'])
    print("*****")

    store_ = config['configurable']['__pregel_runtime'].store
    
    print(store_.search(namespace_for_memory, filter={"context": "Discussing dinner plans"}))
    
    return {"foo": "a", "bar": ["a"]}

def node_b(state: State, config: RunnableConfig):
    return {"foo": "b", "bar": ["b"]}


workflow = StateGraph(State)
workflow.add_node("node_a", node_a)
workflow.add_node("node_b", node_b)
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

In [ ]:
checkpointer = InMemorySaver()
config = {"configurable": {"thread_id": "100",
                           "user_id": 1}}

graph = workflow.compile(checkpointer=checkpointer, store=store)

In [ ]:
graph.invoke({"foo": ""},  config=config)

In [ ]:
Q = graph.nodes['node_a']